In [31]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

RANDOM_STATE = 42

In [2]:
merged_data = pd.read_pickle('covid19_sera_merged.pkl')

In [3]:
# Observe the dataset
merged_data.head()

,maestro_column_unmod,maestro_column_variant,patient_id,label,y_covid,y_severe_type,Patient ID a,Metabolomics ID e,Metabolites,(14 or 15)-methylpalmitate (a17:0 or i17:0),...,variant_ccms_row_101452,variant_ccms_row_101453,variant_ccms_row_101454,variant_ccms_row_101455,variant_ccms_row_101456,variant_ccms_row_101457,variant_ccms_row_101458,variant_ccms_row_101459,variant_ccms_row_101460,variant_ccms_row_101461
0,_dyn_#Healthy.HC1.Healthy..HC1.1_intensity_for...,_dyn_#Healthy.HC1.Healthy..HC1.1_intensity_for...,HC1,Healthy,1,0,HC1,jkdz1,jkdz1,7439425,...,0.0,1.958825,0.00000,0.000000,1.174198,0.0,0.0,0.0,0.370040,0.0
1,_dyn_#Healthy.HC10.Healthy..HC10.1_intensity_f...,_dyn_#Healthy.HC10.Healthy..HC10.1_intensity_f...,HC10,Healthy,1,0,HC10,jkdz10,jkdz10,16636076,...,0.0,2.544654,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
2,_dyn_#Healthy.HC12.Healthy..HC12.1_intensity_f...,_dyn_#Healthy.HC12.Healthy..HC12.1_intensity_f...,HC12,Healthy,1,0,HC12,jkdz12,jkdz12,9140857,...,0.0,0.913199,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
3,_dyn_#Healthy.HC13.Healthy..HC13.1_intensity_f...,_dyn_#Healthy.HC13.Healthy..HC13.1_intensity_f...,HC13,Healthy,1,0,HC13,jkdz13,jkdz13,7863659.5,...,0.0,1.035449,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
4,_dyn_#Healthy.HC17.Healthy..HC17.1_intensity_f...,_dyn_#Healthy.HC17.Healthy..HC17.1_intensity_f...,HC17,Healthy,1,0,HC17,jkdz17,jkdz17,7996507.5,...,0.0,0.110892,0.81455,0.863862,0.000000,0.0,0.0,0.0,0.791905,0.0


In [4]:
def get_unmod_features(df):
    return df.iloc[:, 950:102411]

def get_variant_features(df):
    return df.iloc[:, 102411:]

def get_meta_features(df):
    return df.iloc[:, 9:950]

def get_all_features(df):
    return df.iloc[:, 9:].copy()

def get_y(df):
    return df.iloc[:, :7].copy()

def get_y_covid(df):
    return df.loc[:, "y_covid"]

In [5]:
merged_data_y = get_y(merged_data)

In [6]:
merged_data_y.head()

,maestro_column_unmod,maestro_column_variant,patient_id,label,y_covid,y_severe_type,Patient ID a
0,_dyn_#Healthy.HC1.Healthy..HC1.1_intensity_for...,_dyn_#Healthy.HC1.Healthy..HC1.1_intensity_for...,HC1,Healthy,1,0,HC1
1,_dyn_#Healthy.HC10.Healthy..HC10.1_intensity_f...,_dyn_#Healthy.HC10.Healthy..HC10.1_intensity_f...,HC10,Healthy,1,0,HC10
2,_dyn_#Healthy.HC12.Healthy..HC12.1_intensity_f...,_dyn_#Healthy.HC12.Healthy..HC12.1_intensity_f...,HC12,Healthy,1,0,HC12
3,_dyn_#Healthy.HC13.Healthy..HC13.1_intensity_f...,_dyn_#Healthy.HC13.Healthy..HC13.1_intensity_f...,HC13,Healthy,1,0,HC13
4,_dyn_#Healthy.HC17.Healthy..HC17.1_intensity_f...,_dyn_#Healthy.HC17.Healthy..HC17.1_intensity_f...,HC17,Healthy,1,0,HC17


In [7]:
merged_data_X = get_all_features(merged_data)

In [8]:
merged_data_X.head()

,(14 or 15)-methylpalmitate (a17:0 or i17:0),(16 or 17)-methylstearate (a19:0 or i19:0),(2 or 3)-decenoate (10:1n7 or n8),"(2,4 or 2,5)-dimethylphenol sulfate",(R)-3-hydroxybutyrylcarnitine,(S)-3-hydroxybutyrylcarnitine,(S)-a-amino-omega-caprolactam,1-(1-enyl-oleoyl)-GPE (P-18:1)*,1-(1-enyl-palmitoyl)-2-arachidonoyl-GPC (P-16:0/20:4)*,1-(1-enyl-palmitoyl)-2-arachidonoyl-GPE (P-16:0/20:4)*,...,variant_ccms_row_101452,variant_ccms_row_101453,variant_ccms_row_101454,variant_ccms_row_101455,variant_ccms_row_101456,variant_ccms_row_101457,variant_ccms_row_101458,variant_ccms_row_101459,variant_ccms_row_101460,variant_ccms_row_101461
0,7439425,1007010.0625,832046.25,79229.4219,NaN,NaN,NaN,1986184.875,44282120,21753136,...,0.0,1.958825,0.00000,0.000000,1.174198,0.0,0.0,0.0,0.370040,0.0
1,16636076,1793032.25,447215.5625,326591.0938,NaN,NaN,757049.25,1187651.25,28710652,12963986,...,0.0,2.544654,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
2,9140857,1179147.375,443550.2188,198629.5156,NaN,NaN,915471.5,781512,20551872,7749758.5,...,0.0,0.913199,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
3,7863659.5,1027411.125,838084.8125,49912.3047,NaN,NaN,305460.7813,479870.2813,17261926,6189067,...,0.0,1.035449,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
4,7996507.5,823444.4375,279286.1875,134257.1563,208471.2188,334848.4063,676507.4375,1719241,24612552,12316496,...,0.0,0.110892,0.81455,0.863862,0.000000,0.0,0.0,0.0,0.791905,0.0


### Handling missing values
###### How many columns have missing values?

In [9]:
(merged_data_X.isnull().sum() > 0).value_counts()

False    202595
True       1268
dtype: int64

##### Mean imputation for missing values

In [10]:
for i in merged_data_X.columns[merged_data_X.isnull().any(axis = 0)]:     #---Applying Only on variables with NaN values
    merged_data_X[i].fillna(merged_data_X[i].mean(), inplace = True)

##### How many columns still have NaN after mean imputation? 
##### Let us drop all these columns containing all NaN values

In [11]:
(merged_data_X.isnull().sum() > 0).value_counts()

False    203129
True        734
dtype: int64

In [12]:
merged_data_X.dropna(axis = 1, how = 'all', inplace = True)

##### No more NaN values in the dataset

In [13]:
merged_data_X.isnull().values.any(), (merged_data_X.isnull().sum() > 0).value_counts()

(False,
 False    203129
 dtype: int64)

# Problem: Covid v/s Non-Covid
## Experiment with Dimensionality Reduction techniques
### Based on Pisit's initial analysis with Random Forest and Boosted trees we saw roughly around 75-85 features being used - rest were all dropped. For that reason I choose to reduce our data dimensions to total of 100 (for all types of data), 45 for unmod pep, 45 for variant peptide and 10 for metabolomics data

In [24]:
def get_splits(X_dataset_df, y):
    # Prepare our data for analysis - keep distribution of classes same in all the sets
    X_train, X_test_, y_train, y_test_ = train_test_split(X_dataset_df, y, test_size = 0.3, random_state = RANDOM_STATE, stratify = y)
    X_test, X_val, y_test, y_val = train_test_split(X_test_, y_test_, test_size = 0.5, random_state = RANDOM_STATE, stratify = y_test_)
    
    print(y_train.value_counts(), y_val.value_counts(), y_test.value_counts(), end="\n")
        
    return X_train, y_train, X_val, y_val, X_test, y_test

def train_and_eval(X_train, y_train, X_val, y_val, X_test, y_test):
    model = SVC()
    
    model.fit(X_train, y_train)
    
    print(accuracy_score(model.predict(X_train), y_train))
    print(accuracy_score(model.predict(X_val), y_val))
    print(accuracy_score(model.predict(X_test), y_test))
    
    

## 1. PCA (n=50)
### All features

In [33]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
t1 = datetime.now()
reduced_X_pca = pca.fit_transform(merged_data_X)
print(f"Time for PCA is: {(datetime.now() - t1).total_seconds()}s")

X_train_pca, y_train_pca, X_val_pca, y_val_pca, X_test_pca, y_test_pca = get_splits(reduced_X_pca, merged_data_y["y_covid"])

Time for PCA is: 6.851558s
1    41
0    17
Name: y_covid, dtype: int64 1    9
0    4
Name: y_covid, dtype: int64 1    9
0    3
Name: y_covid, dtype: int64


In [34]:
train_and_eval(X_train_pca, y_train_pca, X_val_pca, y_val_pca, X_test_pca, y_test_pca)

0.896551724137931
0.7692307692307693
0.9166666666666666


## 2. Factor Analysis (FA) (n=50)
### All features

In [35]:
from sklearn.decomposition import FactorAnalysis

fa = FactorAnalysis(n_components=50, random_state=RANDOM_STATE)
t1 = datetime.now()
reduced_X_fa = fa.fit_transform(merged_data_X)
print(f"Time for FA is: {(datetime.now() - t1).total_seconds()}s")

X_train_fa, y_train_fa, X_val_fa, y_val_fa, X_test_fa, y_test_fa = get_splits(reduced_X_fa, merged_data_y["y_covid"])

Time for PCA is: 46.463107s
1    41
0    17
Name: y_covid, dtype: int64 1    9
0    4
Name: y_covid, dtype: int64 1    9
0    3
Name: y_covid, dtype: int64


In [36]:
train_and_eval(X_train_fa, y_train_fa, X_val_fa, y_val_fa, X_test_fa, y_test_fa)

0.8793103448275862
0.6923076923076923
0.75


## 3. TruncatedSVD (n=50)
### All features

In [41]:
from sklearn.decomposition import TruncatedSVD

t1 = datetime.now()
svd = TruncatedSVD(n_components=50, algorithm='randomized',random_state=RANDOM_STATE)
reduced_X_svd = svd.fit_transform(merged_data_X)
print(f"Time for TruncatedSVD is: {(datetime.now() - t1).total_seconds()}s")

X_train_svd, y_train_svd, X_val_svd, y_val_svd, X_test_svd, y_test_svd = get_splits(reduced_X_svd, merged_data_y["y_covid"])

Time for TruncatedSVD is: 6.792177s
1    41
0    17
Name: y_covid, dtype: int64 1    9
0    4
Name: y_covid, dtype: int64 1    9
0    3
Name: y_covid, dtype: int64


In [42]:
train_and_eval(X_train_svd, y_train_svd, X_val_svd, y_val_svd, X_test_svd, y_test_svd)

0.7241379310344828
0.6923076923076923
0.75


## 4. KernelPCA (n=50)
### All features

In [44]:
from sklearn.decomposition import KernelPCA

kpca = KernelPCA(n_components=50, kernel='rbf', random_state=RANDOM_STATE)
t1 = datetime.now()
reduced_X_kpca = kpca.fit_transform(merged_data_X)
print(f"Time for kPCA is: {(datetime.now() - t1).total_seconds()}s")

X_train_kpca, y_train_kpca, X_val_kpca, y_val_kpca, X_test_kpca, y_test_kpca = get_splits(reduced_X_kpca, merged_data_y["y_covid"])

Time for kPCA is: 5.161817s
1    41
0    17
Name: y_covid, dtype: int64 1    9
0    4
Name: y_covid, dtype: int64 1    9
0    3
Name: y_covid, dtype: int64


In [45]:
train_and_eval(X_train_kpca, y_train_kpca, X_val_kpca, y_val_kpca, X_test_kpca, y_test_kpca)

0.9482758620689655
0.6923076923076923
0.75
